In [1]:
from utils.tools import dotdict
# from exp.exp_informer import Exp_Informer
import torch

In [2]:
args = dotdict()

args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]

args.data = 'ETTh1' # data
args.root_path = '/home/ec2-user/SageMaker/timeseries_practise/ETDataset/ETT-small/' # root path of data file
args.data_path = 'ETTh1.csv' # data file
args.features = 'M' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'OT' # target feature in S or MS task
args.freq = 'h' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.checkpoints = './informer_checkpoints' # location of model checkpoints

args.seq_len = 96 # input sequence length of Informer encoder
args.label_len = 48 # start token length of Informer decoder
args.pred_len = 24 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

args.enc_in = 7 # encoder input size
args.dec_in = 7 # decoder input size
args.c_out = 7 # output size
args.factor = 5 # probsparse attn factor
args.d_model = 512 # dimension of model
args.n_heads = 8 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 2048 # dimension of fcn in model
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0
args.freq = 'h'

args.batch_size = 32 
args.learning_rate = 0.0001
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 1
args.patience = 3
args.des = 'exp'

args.use_gpu = False
args.gpu = 0

args.use_multi_gpu = False
args.devices = '0,1,2,3'


In [3]:
# Set augments by using data name
data_parser = {
    'ETTh1':{'data':'ETTh1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTh2':{'data':'ETTh2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTm1':{'data':'ETTm1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTm2':{'data':'ETTm2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
}
if args.data in data_parser.keys():
    data_info = data_parser[args.data]
    args.data_path = data_info['data']
    args.target = data_info['T']
    args.enc_in, args.dec_in, args.c_out = data_info[args.features]

In [4]:
args.detail_freq = args.freq
args.freq = args.freq[-1:]

In [5]:
from data.data_loader import Dataset_ETT_hour, Dataset_ETT_minute, Dataset_Custom, Dataset_Pred
from exp.exp_basic import Exp_Basic
from models.model import Informer, InformerStack

from utils.tools import EarlyStopping, adjust_learning_rate
from utils.metrics import metric

import numpy as np
import json

import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader

import os
import time

import dist.sm_dist as sm_dist

import warnings
warnings.filterwarnings('ignore')

class Exp_Informer(Exp_Basic):
    def __init__(self, args):
        super(Exp_Informer, self).__init__(args)
    
    def _build_model(self):
        model_dict = {
            'informer':Informer,
            'informerstack':InformerStack,
        }
        
        if self.args.model=='informer' or self.args.model=='informerstack':
            e_layers = self.args.e_layers if self.args.model=='informer' else self.args.s_layers
            model = model_dict[self.args.model](
                self.args.enc_in,
                self.args.dec_in, 
                self.args.c_out, 
                self.args.seq_len, 
                self.args.label_len,
                self.args.pred_len, 
                self.args.factor,
                self.args.d_model, 
                self.args.n_heads, 
                e_layers, # self.args.e_layers,
                self.args.d_layers, 
                self.args.d_ff,
                self.args.dropout, 
                self.args.attn,
                self.args.embed,
                self.args.freq,
                self.args.activation,
                self.args.output_attention,
                self.args.distil,
                self.args.mix,
                self.device
            ).float()
        
        if self.args.use_gpu:
            model = sm_dist.dist_model(model.to(self.device))
            model.cuda(self.args.local_rank)
#             model = nn.DataParallel(model, device_ids=self.args.device_ids)
        return model


    def _get_data(self, flag):
        args = self.args
        data_dict = {
            'ETTh1':Dataset_ETT_hour,
            'ETTh2':Dataset_ETT_hour,
            'ETTm1':Dataset_ETT_minute,
            'ETTm2':Dataset_ETT_minute,
            'WTH':Dataset_Custom,
            'ECL':Dataset_Custom,
            'Solar':Dataset_Custom,
            'custom':Dataset_Custom,
        }
        Data = data_dict[args.data]
        timeenc = 0 if args.embed!='timeF' else 1

        if flag == 'test':
            shuffle_flag = False; drop_last = True; batch_size = args.batch_size; freq=args.freq
        elif flag=='pred':
            shuffle_flag = False; drop_last = False; batch_size = 1; freq=args.detail_freq
            Data = Dataset_Pred
        else:
            shuffle_flag = True; drop_last = True; batch_size = args.batch_size; freq=args.freq
                        
        data_set = Data(
            root_path=args.root_path,
            data_path=args.data_path,
            flag=flag,
            size=[args.seq_len, args.label_len, args.pred_len],
            features=args.features,
            target=args.target,
            inverse=args.inverse,
            timeenc=timeenc,
            freq=freq,
            cols=args.cols
        )
        
        #######################################################
        ####### SageMaker Distributed Data Parallel     #######
        #######  - Add num_replicas and rank            #######
        #######################################################
        train_sampler = None
        
        if self.args.use_gpu:
            if flag == 'train':
                shuffle_flag = False
                train_sampler = torch.utils.data.distributed.DistributedSampler(
                    data_set, num_replicas=args.world_size, rank=args.rank
                ) 
        #######################################################
        print(flag, len(data_set))
        data_loader = DataLoader(
            data_set,
            batch_size=batch_size,
            shuffle=shuffle_flag,
            num_workers=args.num_workers,
            sampler=train_sampler,
            drop_last=drop_last)

        return data_set, data_loader

    def _select_optimizer(self):
        model_optim = optim.Adam(self.model.parameters(), lr=self.args.learning_rate)
        return model_optim
    
    def _select_criterion(self):
        criterion =  nn.MSELoss()
        return criterion

    def vali(self, vali_data, vali_loader, criterion):
        self.model.eval()
        total_loss = []
        for i, (batch_x,batch_y,batch_x_mark,batch_y_mark) in enumerate(vali_loader):
            pred, true = self._process_one_batch(
                vali_data, batch_x, batch_y, batch_x_mark, batch_y_mark)
            loss = criterion(pred.detach().cpu(), true.detach().cpu())
            total_loss.append(loss)
        total_loss = np.average(total_loss)
        self.model.train()
        return total_loss

    def train(self, setting):
        train_data, train_loader = self._get_data(flag = 'train')
        vali_data, vali_loader = self._get_data(flag = 'val')
        test_data, test_loader = self._get_data(flag = 'test')

        path = os.path.join(self.args.checkpoints, setting)

        if not os.path.exists(path) and self.args.local_rank==0:
            os.makedirs(path)

        time_now = time.time()
        
        train_steps = len(train_loader)
        early_stopping = EarlyStopping(patience=self.args.patience, verbose=True)
        
        model_optim = self._select_optimizer()
        criterion =  self._select_criterion()

        if self.args.use_amp:
            scaler = torch.cuda.amp.GradScaler()

        for epoch in range(self.args.train_epochs):
            iter_count = 0
            train_loss = []
            
            self.model.train()
            epoch_time = time.time()
            for i, (batch_x,batch_y,batch_x_mark,batch_y_mark) in enumerate(train_loader):
                iter_count += 1
                
                model_optim.zero_grad()
                pred, true = self._process_one_batch(
                    train_data, batch_x, batch_y, batch_x_mark, batch_y_mark)
                loss = criterion(pred, true)
                train_loss.append(loss.item())
                
                if (i+1) % 100==0:
                    print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
                    speed = (time.time()-time_now)/iter_count
                    left_time = speed*((self.args.train_epochs - epoch)*train_steps - i)
                    print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
                    iter_count = 0
                    time_now = time.time()
                
                if self.args.use_amp:
                    scaler.scale(loss).backward()
                    scaler.step(model_optim)
                    scaler.update()
                else:
                    loss.backward()
                    model_optim.step()

            print("Epoch: {} cost time: {}".format(epoch+1, time.time()-epoch_time))
            train_loss = np.average(train_loss)
            vali_loss = self.vali(vali_data, vali_loader, criterion)
            test_loss = self.vali(test_data, test_loader, criterion)

            print("Epoch: {0}, Steps: {1} | Train Loss: {2:.7f}, Valid Loss: {3:.7f}, Test Loss: {4:.7f},".format(
                epoch + 1, train_steps, train_loss, vali_loss, test_loss))
            early_stopping(vali_loss, self.model, path)
            if early_stopping.early_stop:
                print("Early stopping")
                break

            adjust_learning_rate(model_optim, epoch+1, self.args)
        
        
        
        best_model_path = path+'/'+'checkpoint.pth'
        if self.args.use_multi_gpu and self.args.use_gpu:
            sm_dist.barrier()
        self.model.load_state_dict(torch.load(best_model_path))
        
        return self.model

    def test(self, setting):
        test_data, test_loader = self._get_data(flag='test')
        
        self.model.eval()
        
        preds = []
        trues = []
        
        for i, (batch_x,batch_y,batch_x_mark,batch_y_mark) in enumerate(test_loader):
            pred, true = self._process_one_batch(
                test_data, batch_x, batch_y, batch_x_mark, batch_y_mark)
            preds.append(pred.detach().cpu().numpy())
            trues.append(true.detach().cpu().numpy())

        preds = np.array(preds)
        trues = np.array(trues)
#         print('test shape:', preds.shape, trues.shape)
        preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])
        trues = trues.reshape(-1, trues.shape[-2], trues.shape[-1])
#         print('test shape:', preds.shape, trues.shape)

        # result save
        folder_path = os.path.join(self.args.checkpoints, 'results/' + setting +'/')
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        mae, mse, rmse, mape, mspe = metric(preds, trues)
        print('mse:{}, mae:{}'.format(mse, mae))

        np.save(folder_path+'metrics.npy', np.array([mae, mse, rmse, mape, mspe]))
        np.save(folder_path+'pred.npy', preds)
        np.save(folder_path+'true.npy', trues)

        report_dict = {"mse" : mse, "mae": mae}
        

        report_dict = {
            "test_metrics": {
                "mse": {
                    "value": float(mse)
                },
                "mae": {
                    "value": float(mae)
                },
            }
        }
        
        test_output_path = self.args.checkpoints+"/test_report.json"
        print("Saving classification report to {}".format(test_output_path))

        with open(test_output_path, "w") as f:
            f.write(json.dumps(report_dict))

        return

    def predict(self, setting, load=False):
        pred_data, pred_loader = self._get_data(flag='pred')
        
        if load:
            path = os.path.join(self.args.checkpoints, setting)
            best_model_path = path+'/'+'checkpoint.pth'
            self.model.load_state_dict(torch.load(best_model_path))

        self.model.eval()
        
        preds = []
        
        for i, (batch_x,batch_y,batch_x_mark,batch_y_mark) in enumerate(pred_loader):
            pred, true = self._process_one_batch(
                pred_data, batch_x, batch_y, batch_x_mark, batch_y_mark)
            preds.append(pred.detach().cpu().numpy())

        preds = np.array(preds)
        preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])
        
        # result save
        folder_path = os.path.join(self.args.checkpoints, 'results/' + setting +'/')
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        
        np.save(folder_path+'real_prediction.npy', preds)
        
        return

    def _process_one_batch(self, dataset_object, batch_x, batch_y, batch_x_mark, batch_y_mark):
        batch_x = batch_x.float().to(self.device)
        batch_y = batch_y.float()

        batch_x_mark = batch_x_mark.float().to(self.device)
        batch_y_mark = batch_y_mark.float().to(self.device)

        # decoder input
        if self.args.padding==0:
            dec_inp = torch.zeros([batch_y.shape[0], self.args.pred_len, batch_y.shape[-1]]).float()
        elif self.args.padding==1:
            dec_inp = torch.ones([batch_y.shape[0], self.args.pred_len, batch_y.shape[-1]]).float()
        dec_inp = torch.cat([batch_y[:,:self.args.label_len,:], dec_inp], dim=1).float().to(self.device)
        # encoder - decoder
        if self.args.use_amp:
            with torch.cuda.amp.autocast():
                if self.args.output_attention:
                    outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                else:
                    outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
        else:
            if self.args.output_attention:
                outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
            else:
                outputs = self.model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
        if self.args.inverse:
            outputs = dataset_object.inverse_transform(outputs)
        f_dim = -1 if self.args.features=='MS' else 0
        batch_y = batch_y[:,-self.args.pred_len:,f_dim:].to(self.device)

        return outputs, batch_y


NO SageMaker DP


In [6]:
data_dict = {
    'ETTh1':Dataset_ETT_hour,
    'ETTh2':Dataset_ETT_hour,
    'ETTm1':Dataset_ETT_minute,
    'ETTm2':Dataset_ETT_minute,
    'WTH':Dataset_Custom,
    'ECL':Dataset_Custom,
    'Solar':Dataset_Custom,
    'custom':Dataset_Custom,
}
Data = data_dict[args.data]
timeenc = 0 if args.embed!='timeF' else 1


shuffle_flag = False; drop_last = False; batch_size = 1; freq=args.detail_freq
Data = Dataset_Pred

In [7]:
root_path=args.root_path
data_path=args.data_path
flag="pred"
size=[args.seq_len, args.label_len, args.pred_len]
features=args.features
target=args.target
inverse=args.inverse
timeenc=timeenc
freq=freq
cols=args.cols

In [8]:
seq_len = size[0]
label_len = size[1]
pred_len = size[2]

In [9]:
import os
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
# from sklearn.preprocessing import StandardScaler

from utils.tools import StandardScaler
from utils.timefeatures import time_features

In [10]:
scaler = StandardScaler()

In [181]:
df_raw = pd.read_csv("../Informer2020/predict_data_re.txt")

In [183]:
border1 = len(df_raw)-seq_len
border2 = len(df_raw)

cols_data = df_raw.columns[1:]
df_data = df_raw[cols_data]

In [214]:
df_data

,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,6.831999778747560,3.617000102996830,4.76200008392334,1.9539999961853000,2.497999906539920,1.1269999742507900,6.683000087738040
1,5.357999801635740,3.95199990272522,2.45199990272522,2.0969998836517300,2.924000024795530,1.371000051498410,6.401999950408940
2,5.692999839782720,3.3489999771118200,2.131999969482420,1.9900000095367400,3.13700008392334,1.4919999837875400,6.752999782562260
3,5.223999977111820,2.1429998874664300,2.4159998893737800,1.1019999980926500,2.4670000076293900,1.2489999532699600,3.6579999923706100
4,5.425000190734860,3.0810000896453900,2.559000015258790,1.350000023841860,2.497999906539920,1.156999945640560,3.6579999923706100
...,...,...,...,...,...,...,...
426,-1.6740000247955300,3.5499999523162800,-5.614999771118160,2.131999969482420,3.4719998836517300,1.5230000019073500,10.904000282287600
427,-5.492000102996830,4.287000179290770,-9.131999969482420,2.2739999294281000,3.532999992370610,1.6749999523162800,11.043999671936000
428,2.812999963760380,3.818000078201290,-0.8169999718666080,2.0969998836517300,3.7160000801086400,1.5230000019073500,10.270999908447300
429,9.243000030517580,3.818000078201290,5.4720001220703100,2.0969998836517300,3.6549999713897700,1.4320000410080000,9.777999877929690


In [180]:
df_raw = pd.read_csv("../Informer2020/predict_data_re.txt")

# cols = list(df_raw.columns); cols.remove(target); cols.remove('date')
# cols

# df_raw = df_raw[['date']+cols+[target]]
# df_raw

border1 = len(df_raw)-seq_len
border2 = len(df_raw)

cols_data = df_raw.columns[1:]
df_data = df_raw[cols_data]

scaler.fit(df_data.values)
data = scaler.transform(df_data.values)

tmp_stamp = df_raw[['date']][border1:border2]

tmp_stamp['date'] = pd.to_datetime(tmp_stamp.date)

pred_dates = pd.date_range(tmp_stamp.date.values[-1], periods=pred_len+1, freq=freq)

df_stamp = pd.DataFrame(columns = ['date'])
df_stamp.date = list(tmp_stamp.date.values) + list(pred_dates[1:])
data_stamp = time_features(df_stamp, timeenc=timeenc, freq=freq[-1:])

data_x = data[border1:border2]
data_y = data[border1:border2]
data_stamp = data_stamp

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [13]:
s_begin = 0
s_end = s_begin + seq_len
r_begin = s_end - label_len
r_end = r_begin + label_len + pred_len

seq_x = data_x[s_begin:s_end]

seq_y = data_y[r_begin:r_begin+label_len]
seq_x_mark = data_stamp[s_begin:s_end]
seq_y_mark = data_stamp[r_begin:r_end]

In [14]:
seq_x.shape

(96, 7)

In [15]:
config_file='../model/results/informer_ETTh1_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtFalse_mxFalse_exp_0/training_config.npy'

In [16]:
config = np.load(config_file, allow_pickle=True)
args = config.tolist()

In [17]:
def _process_one_batch(dataset_object, batch_x, batch_y, batch_x_mark, batch_y_mark, model):
    batch_x = batch_x.float().to(device)
    batch_y = batch_y.float()

    batch_x_mark = batch_x_mark.float().to(device)
    batch_y_mark = batch_y_mark.float().to(device)

    # decoder input
    if args.padding==0:
        dec_inp = torch.zeros([batch_y.shape[0], args.pred_len, batch_y.shape[-1]]).float()
    elif args.padding==1:
        dec_inp = torch.ones([batch_y.shape[0], args.pred_len, batch_y.shape[-1]]).float()
    dec_inp = torch.cat([batch_y[:,:args.label_len,:], dec_inp], dim=1).float().to(device)
    # encoder - decoder
    if args.use_amp:
        with torch.cuda.amp.autocast():
            if args.output_attention:
                outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
            else:
                outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
    else:
        if args.output_attention:
            outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
        else:
            outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
    if args.inverse:
        outputs = dataset_object.inverse_transform(outputs)
    f_dim = -1 if args.features=='MS' else 0
    batch_y = batch_y[:,-args.pred_len:,f_dim:].to(device)

    return outputs, batch_y

In [18]:
model_dir='../model/'
device='cpu'

In [19]:
model_dict = {
    'informer':Informer,
    'informerstack':InformerStack,
}

if args.model=='informer' or args.model=='informerstack':
    e_layers = args.e_layers if args.model=='informer' else args.s_layers
    model = model_dict[args.model](
        args.enc_in,
        args.dec_in, 
        args.c_out, 
        args.seq_len, 
        args.label_len,
        args.pred_len, 
        args.factor,
        args.d_model, 
        args.n_heads, 
        e_layers, # self.args.e_layers,
        args.d_layers, 
        args.d_ff,
        args.dropout, 
        args.attn,
        args.embed,
        args.freq,
        args.activation,
        args.output_attention,
        args.distil,
        args.mix,
        device
    ).float()

# with open(os.path.join(model_dir, args.setting, "checkpoint.pth"), 'rb') as f:
#     model.load_state_dict(torch.load(f))

print("Informer Model loaded")
model.eval()
model = model.to(device)

Informer Model loaded


In [20]:
data_dict = {
    'ETTh1':Dataset_ETT_hour,
    'ETTh2':Dataset_ETT_hour,
    'ETTm1':Dataset_ETT_minute,
    'ETTm2':Dataset_ETT_minute,
    'WTH':Dataset_Custom,
    'ECL':Dataset_Custom,
    'Solar':Dataset_Custom,
    'custom':Dataset_Custom,
}
Data = data_dict[args.data]
timeenc = 0 if args.embed!='timeF' else 1

shuffle_flag = False; drop_last = False; batch_size = 1; freq=args.detail_freq
Data = Dataset_Pred

In [21]:
Data=Dataset_Pred

In [22]:
pred_data = Data(
    root_path="./",
    data_path="../Informer2020/predict_test.csv",
    flag="pred",
    size=[args.seq_len, args.label_len, args.pred_len],
    features=args.features,
    target="OT\n",
    inverse=args.inverse,
    timeenc=timeenc,
    freq=freq,
    cols=args.cols
)

df_raw.columns : Index(['date', 'HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OT\n'], dtype='object')
self.target : OT



In [23]:
pred_loader = DataLoader(
    pred_data,
    batch_size=batch_size,
    shuffle=shuffle_flag,
    num_workers=args.num_workers,
    sampler=None,
    drop_last=drop_last)      

#     pred_data = res['data_set']
#     pred_loader = res['data_loader']
preds = []

In [46]:
r_begin.item()

48

In [164]:
preds = []
trues = []
for i, (batch_x,batch_y,batch_x_mark,batch_y_mark, r_begin) in enumerate(pred_loader):
    print(batch_x.shape,batch_y.shape,batch_x_mark.shape,batch_y_mark.shape,r_begin)
    pred, true = _process_one_batch(
        pred_data, batch_x, batch_y, batch_x_mark, batch_y_mark, model)
    
    pred = pred_data.scaler.inverse_transform(pred)
#     true = pred_data.scaler.inverse_transform(true)
    preds.append(pred.detach().cpu().numpy())
#     trues.append(true.detach().cpu().numpy())
    if i == 0:
        start_point = r_begin.item()

    end_point = r_begin.item()+pred_data.label_len
    
preds = np.array(preds)
preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])    

prediction = preds[0,:,-1]

# trues = np.array(trues)
# trues = trues.reshape(-1, trues.shape[-2], trues.shape[-1])    

# gt = trues[0,:,-1]

pred_timestamp = pred_data.df_stamp
pred_prediction = pred_data.prediction_data

pred_dataset = pd.concat([pred_timestamp, pred_prediction], axis=1)
pred_dataset = pred_dataset.reset_index()
pred_dataset.drop('index', axis=1, inplace=True)

start=pred_dataset.shape[0]-args.pred_len
pred_result=pred_dataset.loc[start:].reset_index()
final_result = pd.concat([pred_result, pd.DataFrame(prediction, columns=['Prediction'])], axis=1)
final_result.drop('index', axis=1, inplace=True)

final_result.to_csv("./prediciton_result.csv")

In [29]:
exp = Exp_Informer

In [30]:
data_dict = {
    'ETTh1':Dataset_ETT_hour,
    'ETTh2':Dataset_ETT_hour,
    'ETTm1':Dataset_ETT_minute,
    'ETTm2':Dataset_ETT_minute,
    'WTH':Dataset_Custom,
    'ECL':Dataset_Custom,
    'Solar':Dataset_Custom,
    'custom':Dataset_Custom,
}

In [31]:
import pandas as pd

In [32]:
df_raw = pd.read_csv("../Informer2020/predict_test.csv")
args.target='OT\n'

In [33]:
if args.cols:
    cols=args.cols.copy()
    cols.remove(args.target)
else:
    cols = list(df_raw.columns); cols.remove(args.target); cols.remove('date')
df_raw = df_raw[['date']+cols+[args.target]]

border1 = len(df_raw)-args.seq_len
border2 = len(df_raw)

In [34]:
if args.features=='M' or args.features=='MS':
    cols_data = df_raw.columns[1:]
    df_data = df_raw[cols_data]
elif args.features=='S':
    df_data = df_raw[[args.target]]

In [35]:
data = df_data.values

In [36]:
tmp_stamp = df_raw[['date']][border1:border2]
tmp_stamp['date'] = pd.to_datetime(tmp_stamp.date)
pred_dates = pd.date_range(tmp_stamp.date.values[-1], periods=args.pred_len+1, freq=args.freq)

In [37]:
df_stamp = pd.DataFrame(columns = ['date'])
df_stamp.date = list(tmp_stamp.date.values) + list(pred_dates[1:])
data_stamp = time_features(df_stamp, timeenc=timeenc, freq=freq[-1:])

In [38]:
data_x.shape, data_y.shape

((96, 7), (96, 7))

In [39]:
args.label_len, args.pred_len

(48, 24)

In [40]:
index=0

In [41]:
s_begin = index
s_end = s_begin + args.seq_len
r_begin = s_end - args.label_len
r_end = r_begin + args.label_len + args.pred_len

In [42]:
s_begin, s_end, r_begin, r_end

(0, 96, 48, 120)

In [43]:
seq_x = data_x[s_begin:s_end]

In [44]:
seq_x = data_x[s_begin:s_end]
if args.inverse:
    seq_y = data_x[r_begin:r_begin+args.label_len]

else:
    seq_y = data_y[r_begin:r_begin+args.label_len]

seq_x_mark = data_stamp[s_begin:s_end]
seq_y_mark = data_stamp[r_begin:r_end]


In [45]:
for i, (batch_x,batch_y,batch_x_mark,batch_y_mark,r_begin) in enumerate(pred_loader):
    pred, true = _process_one_batch(
        pred_data, batch_x, batch_y, batch_x_mark, batch_y_mark, model)
    if i == 0:
        start_point = r_begin.item()

    end_point = r_begin.item()+pred_data.label_len
    

In [46]:
pred_data.prediction_data

,date
11,2018-06-22 20:00:00
12,2018-06-22 21:00:00
13,2018-06-22 22:00:00
14,2018-06-22 23:00:00
15,2018-06-23 00:00:00
...,...
102,2018-06-26 15:00:00
103,2018-06-26 16:00:00
104,2018-06-26 17:00:00
105,2018-06-26 18:00:00


In [48]:
pred_data.prediction_data

,date
11,2018-06-22 20:00:00
12,2018-06-22 21:00:00
13,2018-06-22 22:00:00
14,2018-06-22 23:00:00
15,2018-06-23 00:00:00
...,...
102,2018-06-26 15:00:00
103,2018-06-26 16:00:00
104,2018-06-26 17:00:00
105,2018-06-26 18:00:00


In [44]:
pd.concat([pred_data.df_stamp,pred_data.prediction_data], axis=1)

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT\n
0,2018-06-22 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-06-22 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-06-22 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-06-22 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-06-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
115,2018-06-27 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,2018-06-27 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,2018-06-27 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,2018-06-27 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
pred_timestamp = pred_data.df_stamp[start_point:end_point]['date']
pred_timestamp = pred_timestamp.reset_index()['date']

In [76]:
seq_y_df=pd.DataFrame(pred_data.data_x[start_point:end_point], columns=pred_data.cols+[pred_data.target])

In [77]:
pred_array=pd.concat([pred_timestamp,seq_y_df], axis=1)

In [54]:
preds = np.array(preds)
preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1])    

prediction = preds[0,:,-1]

print(f"prediction : {prediction}")

# f = open("informer_result.csv", 'a')
# f.write(data_dir)
# f.close()

IndexError: tuple index out of range

In [340]:
pred_data.append(prediction)

TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

In [ ]:
model_dict = {
    'informer':Informer,
    'informerstack':InformerStack,
}

if args.model=='informer' or args.model=='informerstack':
    e_layers = args.e_layers if args.model=='informer' else args.s_layers
    model = model_dict[args.model](
        args.enc_in,
        args.dec_in, 
        args.c_out, 
        args.seq_len, 
        args.label_len,
        args.pred_len, 
        args.factor,
        args.d_model, 
        args.n_heads, 
        e_layers, # self.args.e_layers,
        args.d_layers, 
        args.d_ff,
        args.dropout, 
        args.attn,
        args.embed,
        args.freq,
        args.activation,
        args.output_attention,
        args.distil,
        args.mix,
    ).float()

In [ ]:
args.setting

In [ ]:
path='./utils/'

In [ ]:
for (root, dirs, files) in os.walk(path):
    print(f"root : {root}, dir : {dirs}, files : {files}")

In [146]:
model_dir='../model/'

In [147]:
args.setting='informer_ETTh1_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtFalse_mxFalse_exp_0'

In [148]:
config_file='../model/results/informer_ETTh1_ftM_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtFalse_mxFalse_exp_0/training_config.npy'

In [149]:
config = np.load(config_file, allow_pickle=True)
args = config.tolist()

In [136]:
def _process_one_batch(dataset_object, batch_x, batch_y, batch_x_mark, batch_y_mark):
    device = 'cpu'
    batch_x = batch_x.float().to(device)
    batch_y = batch_y.float()

    batch_x_mark = batch_x_mark.float().to(device)
    batch_y_mark = batch_y_mark.float().to(device)

    # decoder input
    if args.padding==0:
        dec_inp = torch.zeros([batch_y.shape[0], args.pred_len, batch_y.shape[-1]]).float()
    elif self.args.padding==1:
        dec_inp = torch.ones([batch_y.shape[0], args.pred_len, batch_y.shape[-1]]).float()
    dec_inp = torch.cat([batch_y[:,:args.label_len,:], dec_inp], dim=1).float().to(device)
    # encoder - decoder
    if args.use_amp:
        with torch.cuda.amp.autocast():
            if args.output_attention:
                outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
            else:
                outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
    else:
        if args.output_attention:
            outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
        else:
            outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
    if args.inverse:
        outputs = dataset_object.inverse_transform(outputs)
    f_dim = -1 if args.features=='MS' else 0
    batch_y = batch_y[:,-args.pred_len:,f_dim:].to(device)

    return outputs, batch_y

In [137]:
model_dict = {
    'informer':Informer,
    'informerstack':InformerStack,
}

if args.model=='informer' or args.model=='informerstack':
    e_layers = args.e_layers if args.model=='informer' else args.s_layers
    model = model_dict[args.model](
        args.enc_in,
        args.dec_in, 
        args.c_out, 
        args.seq_len, 
        args.label_len,
        args.pred_len, 
        args.factor,
        args.d_model, 
        args.n_heads, 
        e_layers, # self.args.e_layers,
        args.d_layers, 
        args.d_ff,
        args.dropout, 
        args.attn,
        args.embed,
        args.freq,
        args.activation,
        args.output_attention,
        args.distil,
        args.mix,
    ).float()

with open(os.path.join(model_dir, args.setting, "checkpoint.pth"), 'rb') as f:
    model.load_state_dict(torch.load(f))

print("Net loaded")
model.eval()
model = model.to("cpu")

RuntimeError: Error(s) in loading state_dict for Informer:
	Missing key(s) in state_dict: "enc_embedding.value_embedding.tokenConv.weight", "enc_embedding.value_embedding.tokenConv.bias", "enc_embedding.position_embedding.pe", "enc_embedding.temporal_embedding.embed.weight", "enc_embedding.temporal_embedding.embed.bias", "dec_embedding.value_embedding.tokenConv.weight", "dec_embedding.value_embedding.tokenConv.bias", "dec_embedding.position_embedding.pe", "dec_embedding.temporal_embedding.embed.weight", "dec_embedding.temporal_embedding.embed.bias", "encoder.attn_layers.0.attention.query_projection.weight", "encoder.attn_layers.0.attention.query_projection.bias", "encoder.attn_layers.0.attention.key_projection.weight", "encoder.attn_layers.0.attention.key_projection.bias", "encoder.attn_layers.0.attention.value_projection.weight", "encoder.attn_layers.0.attention.value_projection.bias", "encoder.attn_layers.0.attention.out_projection.weight", "encoder.attn_layers.0.attention.out_projec

In [138]:
import pandas as pd

In [35]:
df_raw = pd.read_csv('../ETDataset/ETT-small/ETTh1_small.csv',skiprows=1 )
target="OT"

In [41]:
df_raw.head(2)

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2018-06-22 10:00:00,5.358,3.952,2.452,2.097,2.924,1.371,6.402
1,2018-06-22 11:00:00,5.693,3.349,2.132,1.990,3.137,1.492,6.753


In [39]:
header=['date','HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OT']

In [40]:
df_raw.columns=header

In [24]:
print(f"df_raw.columns : {df_raw.columns}")
print(f"self.target : {target}")
cols = list(df_raw.columns); cols.remove(target); cols.remove('date')

df_raw.columns : Index(['date', 'HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OT'], dtype='object')
self.target : OT


In [31]:
if None:
    print("t")
else:
    print('g')

g


In [34]:
df_raw[2:]

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
2,2018-06-22 11:00:00,5.693,3.349,2.132,1.990,3.137,1.492,6.753
3,2018-06-22 12:00:00,5.224,2.143,2.416,1.102,2.467,1.249,3.658
4,2018-06-22 13:00:00,5.425,3.081,2.559,1.350,2.498,1.157,3.658
5,2018-06-22 14:00:00,1.808,2.478,-0.462,1.315,2.071,1.097,4.150
6,2018-06-22 15:00:00,1.541,3.014,-0.675,0.888,2.498,1.218,4.643
...,...,...,...,...,...,...,...,...
102,2018-06-26 15:00:00,-1.674,3.550,-5.615,2.132,3.472,1.523,10.904
103,2018-06-26 16:00:00,-5.492,4.287,-9.132,2.274,3.533,1.675,11.044
104,2018-06-26 17:00:00,2.813,3.818,-0.817,2.097,3.716,1.523,10.271
105,2018-06-26 18:00:00,9.243,3.818,5.472,2.097,3.655,1.432,9.778


In [25]:
header=['HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OT']

['HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL']

In [27]:
df_raw = df_raw[['date']+cols+[target]]

In [29]:
cols_data = df_raw.columns[1:]

In [30]:
cols_data

Index(['HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OT'], dtype='object')

In [15]:
if self.cols:
    cols=cols.copy()
    cols.remove(self.target)
else:
    print(f"df_raw.columns : {df_raw.columns}")
    print(f"self.target : {self.target}")
    cols = list(df_raw.columns); cols.remove(self.target); cols.remove('date')
df_raw = df_raw[['date']+cols+[self.target]]

NameError: name 'self' is not defined

In [ ]:
print(f"df_raw.columns : {df_raw.columns}")
print(f"self.target : {self.target}")
cols = list(df_raw.columns); cols.remove(self.target); cols.remove('date')